In [1]:
%pylab
norm = linalg.norm
import cmath
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor
fig=figure() 
ax= fig.add_subplot()

def init_axes():
    ax.set_xlim(-50, 50)
    ax.set_ylim(-50, 50)
    # Change major ticks to show every 20.
    ax.xaxis.set_major_locator(MultipleLocator(10))
    ax.yaxis.set_major_locator(MultipleLocator(10))

    # Change minor ticks to show every 5. (20/4 = 5)
    ax.xaxis.set_minor_locator(MultipleLocator(5))
    ax.yaxis.set_minor_locator(MultipleLocator(5))
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')





Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
class CatchEvents:
    def __init__(self):
        self.catch_events = True
    def isActivated(self):
        return self.catch_events
    def activate(self):
        self.catch_events = True
    def deactivate(self):
        self.catch_events = False
        
class Line2D(CatchEvents):
    def __init__(self,ax, range_val = 50, step = 0.01, draw_margin = True):
        super().__init__()
        ax.clear()
        init_axes()
        self.handles = []
        self.handle_point = None
        self.ax = ax
        self.range_val = range_val
        self.draw_margin = draw_margin
        self.step = step
        
        self.clf_svm = svm.LinearSVC()
        self.clf_svm_gradient = SGDClassifier(learning_rate = 'constant',eta0 = 0.8)
        self.clf_regress = LinearRegression()
        self.clf_ransac= RANSACRegressor()
        self.display = "-"
        self.init()
    def init(self):
        self.clear()
        A,B,C = 1,1, -norm([2,2])**2
        self.graph(A, B, C)
    def getN(self):
        A,B = self.A, self.B
        return np.array([A,B])/norm([A,B])
    def rotate_step(self, anticlockwise = True):
        self.clear()
        if not self.isActivated():
            return
        if anticlockwise:
            rotor = cmath.exp(2 * cmath.pi * self.step * 1j)
        else:
            rotor = cmath.exp(- 2 * cmath.pi * self.step * 1j)
        curr = complex(self.A, self.B)
        z = rotor * curr
        self.graph(A= z.real, B=z.imag, C=self.C)
    def translate_step(self, front = True, step = None):
        if step is None:
            step = self.step
        self.clear()
        A,B,C = self.A, self.B, self.C
        dist = -C/norm([A, B])
        if front:
            dist_plus_delta = dist + step
        else: 
            dist_plus_delta = dist - step
            
        x,y = self.getN() * dist_plus_delta
        C = - A*x - B*y
        self.graph(self.A,self.B, C)
    def scale_step(self, scale_up = False, step = None):
        self.clear()
        A,B,C = self.A, self.B, self.C
        if step is None:
            step = self.step
        if scale_up:
            self.graph(A * step, B * step, C * step)
        else:
            self.graph(A/step, B/step, C/step)
    def rotate(self, x, y):
        self.clear()
        if not self.isActivated():
            return
        A,B = x,y
        self.graph(A,B,self.C)
    def translate(self, x, y):
        self.clear()
        if not self.isActivated():
            return
        A,B,C = self.A, self.B, self.C
        C = - A*x - B*y
        self.graph(self.A, self.B, C)
    def redraw(self):
        init_axes()
        self.clear()
        self.graph(self.A, self.B, self.C)
    def graph(self,A,B,C):
        self.A,self.B,self.C = A,B,C
        self.L_f = lambda x,y: A *x + B *y + C 
        self.dL_f = lambda x,y: (A *x + B *y + C)/np.linalg.norm([A,B])
        R=range(-self.range_val, self.range_val)
        
        normal = np.array([A,B])
        vec_on_line = np.array([-B, A])
        vec_on_line = vec_on_line / np.linalg.norm(vec_on_line)
        
        X,Y,X_plus_1,Y_plus_1,X_minus_1,Y_minus_1 = [],[],[],[],[],[]
        
        dist_from_origin = -C/norm(normal)
        dist_plus_1 = (1-C)/norm(normal)
        dist_minus_1 = (-1-C)/norm(normal)
        
        line_zero = (normal/norm(normal)) * dist_from_origin
        line_plus_1 = (normal/norm(normal)) * dist_plus_1
        line_minus_1 = (normal/norm(normal)) * dist_minus_1
        
        for r in R:
            x,y=vec_on_line* r
            X.append(x + line_zero[0])
            Y.append(y + line_zero[1])
            X_plus_1.append(x + line_plus_1[0])
            Y_plus_1.append(y + line_plus_1[1])
            X_minus_1.append(x + line_minus_1[0])
            Y_minus_1.append(y + line_minus_1[1])
        
        hQuiver = ax.quiver(X[self.range_val],Y[self.range_val], A,B, color = 'r')
        #ax.quiver(X[5],Y[5], vec_on_line[0],vec_on_line[1], color='r')
        hPlot = ax.plot(X, Y, linewidth = 0.5)
        if self.draw_margin:
            hPlot_plus1 = ax.plot(X_plus_1, Y_plus_1,linestyle='--', linewidth = 0.5)
            hPlot_minus1 = ax.plot(X_minus_1, Y_minus_1,linestyle='--', linewidth = 0.5)
            self.handles = [hQuiver, hPlot.pop(0), hPlot_plus1.pop(0), hPlot_minus1.pop(0)]
        else:
            self.handles = [hQuiver, hPlot.pop(0)]
        self.pprint_eq(console = False)
        self.draw_point_on_axes()
        draw()
    def fit(self, X, Y, algo = 'svm'):
        if algo == 'svm':
            self.display = 'SVM'
            self.clf_svm.fit(X,Y)
            self._update(self.clf_svm)
        elif algo == 'gradient_svm':
            for n_iter in range(1000):
                self.clf_svm_gradient.partial_fit(X,Y,classes=unique(Y)) 
            self._update(self.clf_svm_gradient)
        elif algo == 'regression':
            self.display = 'Linear Regression'
            self.clf_regress.fit(X[:,0][:,None],X[:,1])
            self._update_regress(self.clf_regress.coef_[0], self.clf_regress.intercept_)
        elif algo == "ransac":
            self.display = "RANSAC"
            self.clf_ransac.fit(X[:,0][:,None],X[:,1])
            self._update_regress(self.clf_ransac.estimator_.coef_[0],self.clf_ransac.estimator_.intercept_)
    def _update_regress(self, m, b):
        A,B,C = -m, 1,-b
        self.clear()
        self.graph(A, B, C)
    def _update(self, clf):
        A,B = clf.coef_[0]
        C = clf.intercept_[0]
        self.clear()
        self.graph(A, B, C)
    def eval_(self, x, y):
        return self.L_f(x,y)
    def dist(self, x, y):
        return self.dL_f(x,y)
    def clear(self):
        for h in self.handles:
            h.remove()
    def modify_step(self, step):
        self.step = step
    def pprint_eq(self, console = True):
        A,B = self.getN()
        k = norm([self.A, self.B])
        C = self.C/k
        string = f"{round(k,2)} * ({round(A,2)}x + {round(B,2)}y + {round(C,2)}) = 0 -- > {self.display}"
        if console:
            print (string)
        else:
            self.ax.set_title(string)
        
    def draw_point_on_axes(self, x=None, y=None):

        if self.handle_point is not None:
            if x is None:
                x,y = self.handle_point._x, self.handle_point._y
            self.handle_point.remove()
            
        elif x is None:
            return
        dist_val = round(self.dist(x,y),2)
        func_val = round(self.eval_(x,y),2)
        string = f"d = {dist_val}, f = {func_val}"
        self.handle_point = ax.text(x,y,string, fontsize=5)
        draw()        
    
#line2d = Line2D(ax,range_val=50)

In [3]:
class DataPoints(CatchEvents):
    def __init__(self, ax):
        super().__init__()
        init_axes()
        self.ax = ax
        self.points = []
        self.handle = None
        norm_colors = matplotlib.colors.Normalize(vmin=-70, vmax=70, clip=True)
        self.mapper = cm.ScalarMappable(norm=norm_colors, cmap=cm.jet)
    def clear(self):
        self.ax.clear()
        init_axes()
    def redraw_points(self, line2d):
        if not self.isActivated():
            return
        self.clear()
        line2d.redraw()
        for p in self.points:
            x,y,marker = p[0],p[1],p[2]
            if marker == '+' and line2d.dist(x,y) <= 0:
                marker = 'x'
            elif marker == 'o' and line2d.dist(x,y) >= 0:
                marker = 'x'
            self._draw_point(x, y, marker, line2d.dist(x,y))
    def add_point(self, x,y, line2d):  
        if not self.isActivated():
            return
        if  line2d.dist(x,y) > 0:
            self.points.append((x,y,'+'))
            marker = '+'
        elif line2d.dist(x,y) < 0:
            self.points.append((x,y,'o'))
            marker = 'o'
        else:
            self.points.append((x,y,'*'))
            marker = '*'
        self._add_point(x,y,marker, line2d.dist(x,y))
    def get_all(self):
        X, Y = [],[]
        for p in self.points:
            X.append([p[0],p[1]])
            if p[2] == '+':
                Y.append(1)
            else:
                Y.append(0)
        return np.array(X), np.array(Y)
    def _add_point(self,x,y,marker,brightness):
        self.points.append((x,y,marker))
        self._draw_point(x,y, marker, brightness)
    def _draw_point(self, x, y, marker, brightness):
        self.handle=self.ax.scatter(x=x,y=y,marker=marker, color = self.mapper.to_rgba(brightness))   
        draw()
    def clear(self):                          
        if self.handle is not None:
            self.ax.clear()  
            self.handle = None
#data_points = DataPoints(ax)

In [4]:
class EventHandler:
    def __init__(self, ax):
        self.line2d = Line2D(ax,range_val=50)
        self.data_points = DataPoints(ax)
        self.connect()
    def connect(self): 
        self.key = ax.figure.canvas.mpl_connect('key_press_event', self.key_press)
        self.cid = ax.figure.canvas.mpl_connect('button_press_event', self.on_press)
    def key_press(self,event):
        line2d = self.line2d
        data_points = self.data_points
        if event.key == 'r':
            line2d.rotate_step(anticlockwise = True)
        elif event.key == 'R':
            line2d.rotate_step(anticlockwise = False)
        elif event.key == 't':
            line2d.translate_step(front = True, step = 0.5)
        elif event.key == 'T':
            line2d.translate_step(front = False, step = 0.5)
        elif event.key == '+':
            line2d.scale_step(scale_up = True, step = 0.95)
        elif event.key == '-':
            line2d.scale_step(scale_up = False, step = 0.95)
        elif event.key == 'c':
            X,Y = data_points.get_all()
            line2d.fit(X,Y, algo = 'svm')
        elif event.key == 'n':
            X,Y = data_points.get_all()
            line2d.fit(X,Y, algo = 'regression')
        elif event.key == 'N':
            X,Y = data_points.get_all()
            line2d.fit(X,Y, algo = 'ransac')
        data_points.redraw_points(line2d)
    def on_press(self, event):
        line2d = self.line2d
        data_points = self.data_points
        if event.inaxes == ax and int(event.button) == 3:
            x,y = event.xdata, event.ydata
            line2d.draw_point_on_axes(x, y)
        elif event.inaxes == ax:
            x,y = event.xdata, event.ydata
            data_points.add_point(x,y, line2d)
        
eh = EventHandler(ax)



In [ ]:
a=[1]
if not a:
    print ('hi')